In [ ]:
import math
import matplotlib.pyplot as plt
import numpy as np

### DBF Mission 2 Rules Summary & Equations

This script is based on the 2025-26 Design, Build, Fly competition rules for Mission 2: Charter Flight.

#### Payload Rules
- **Passengers**: 2.3-inch rubber ducks (~0.7 oz each).
- **Cargo**: Standard hockey pucks (6 oz each).
- **Payload Ratio**: The airplane must be capable of carrying at least **3 passengers** for each **1 piece of cargo**.
$$N_{\text{cargo}}=\left\lfloor\frac{N_\text{passengers}}3\right\rfloor$$

#### Mission 2 Scoring: Net Income
The score for Mission 2 is the "Net Income" generated from a charter flight within a 5-minute window.

**Net Income Equation:**
$$\text{Net Income}=\text{Income}-\text{Cost}$$

**1. Income Calculation:**
Income is based on the number of passengers/cargo and the number of laps flown.

$$\text{Income}=N_\text{passengers}\times(I_{\text{passenger,fixed}} + (I_{\text{passenger,lap}} \times N_\text{laps})) + N_\text{cargo}\times(I_\text{cargo,fixed} + (I_\text{cargo,lap} \times N_\text{laps}))$$
| Variable | Significance | Value |
| -- | -- | -- |
|$I_\text{passenger,fixed}$|Fixed income per passenger|6|
|$I_\text{passenger,lap}$|Income per passenger per lap|2|
|$I_\text{cargo,fixed}$|Fixed income per cargo|10|
|$I_\text{cargo,lap}$|Income per cargo per lap|8|

**2. Cost Calculation:**
Cost is based on a base operating cost plus costs for passengers/cargo, adjusted by an Efficiency Factor (EF).

$$\text{Cost}=N_\text{laps}\times(C_\text{base,lap} + N_\text{passengers}\times C_\text{passenger,lap} + N_\text{cargo}\times C_\text{cargo,lap})\times e$$
| Variable | Significance | Value |
| -- | -- | -- |
|$C_\text{base,lap}$|Base operating cost per lap|10|
|$C_\text{passenger,lap}$|Passenger operating cost per lap|0.5|
|$C_\text{cargo,lap}$|Cargo operating cost per lap|2|
|$e$|Efficiency Factor This penalizes higher-capacity batteries.|$e=\frac{\text{Total battery capacity}}{100\mathrm{Wh}}$|


In [ ]:
# --- Constants based on the DBF Rules PDF ---

# Passenger (Duck) Properties
DUCK_WEIGHT_OZ = 0.7  # Using max weight for a conservative estimate (0.6-0.7 oz)
DUCK_WIDTH_IN = 2.3
DUCK_LENGTH_IN = 2.5
DUCK_HEIGHT_IN = 2.5
DUCK_VOLUME_IN3 = DUCK_WIDTH_IN * DUCK_LENGTH_IN * DUCK_HEIGHT_IN

# Cargo (Hockey Puck) Properties
PUCK_WEIGHT_OZ = 6.0
PUCK_DIAMETER_IN = 3.0
PUCK_THICKNESS_IN = 1.0
PUCK_VOLUME_IN3 = math.pi * ((PUCK_DIAMETER_IN / 2) ** 2) * PUCK_THICKNESS_IN

# Mission 2 Scoring Parameters
INCOME_FIXED_PER_PASSENGER = 6
INCOME_PER_PASSENGER_PER_LAP = 2
INCOME_FIXED_PER_CARGO = 10
INCOME_PER_CARGO_PER_LAP = 8

COST_BASE_PER_LAP = 10
COST_PER_PASSENGER_PER_LAP = 0.5
COST_PER_CARGO_PER_LAP = 2

In [ ]:
def calculate_payload_properties(num_passengers):
    """
    Calculates the total weight and estimated volume for passengers and associated cargo.

    Args:
        num_passengers (int): The number of passengers (ducks).

    Returns:
        dict: A dictionary containing the number of cargo pieces, total weight in ounces,
              and estimated total volume in cubic inches. Returns None if num_passengers < 3.
    """
    if num_passengers < 3:
        print("Warning: A minimum of 3 passengers is required.")
        # Although the plane must be *capable* of carrying 3, a flight might have 0.
        # For this calculation, we assume we are loading according to rules.
        return {"num_cargo": 0, "weight_oz": num_passengers * DUCK_WEIGHT_OZ, "estimated_volume_in3": num_passengers * DUCK_VOLUME_IN3}

    # Rule: At least 3 passengers for each piece of cargo.
    # This means the max cargo is floor(num_passengers / 3).
    num_cargo = math.floor(num_passengers / 3)

    total_weight_oz = (num_passengers * DUCK_WEIGHT_OZ) + (num_cargo * PUCK_WEIGHT_OZ)

    total_volume_in3 = (num_passengers * DUCK_VOLUME_IN3) + (num_cargo * PUCK_VOLUME_IN3)

    return {"num_cargo": num_cargo, "weight_oz": total_weight_oz, "estimated_volume_in3": total_volume_in3}

In [ ]:
def calculate_mission2_score(num_passengers, num_laps, battery_capacity_wh=100.0):
    """
    Calculates the Net Income for Mission 2 based on the rules.

    Args:
        num_passengers (int): The number of passengers (ducks) flown.
        num_laps (int): The number of laps completed.
        battery_capacity_wh (float, optional): The total propulsion battery capacity
                                              in Watt-hours. Defaults to 100.0.

    Returns:
        float: The calculated Net Income for the mission.
    """
    # The minimum capacity of the airplane is 3 passengers and 1 cargo.
    # However, a team can choose to fly with fewer or no payload.
    if num_passengers > 0:
        payload_props = calculate_payload_properties(num_passengers)
        num_cargo = payload_props["num_cargo"]
    else:  # No passengers means no cargo either for scoring flight
        num_cargo = 0

    # Calculate total income
    income = (num_passengers * (INCOME_FIXED_PER_PASSENGER + (INCOME_PER_PASSENGER_PER_LAP * num_laps))) + (num_cargo * (INCOME_FIXED_PER_CARGO + (INCOME_PER_CARGO_PER_LAP * num_laps)))

    # Calculate total cost
    efficiency_factor = battery_capacity_wh / 100.0
    cost_per_lap = COST_BASE_PER_LAP + (num_passengers * COST_PER_PASSENGER_PER_LAP) + (num_cargo * COST_PER_CARGO_PER_LAP)
    cost = num_laps * cost_per_lap * efficiency_factor

    net_income = income - cost
    return net_income


In [ ]:
example_passengers = 12
example_laps = 5

# 1. Calculate payload weight and volume
payload = calculate_payload_properties(example_passengers)
print(f"--- Payload Properties for {example_passengers} Passengers ---")
print(f"Number of Cargo Pieces: {payload['num_cargo']}")
print(f"Total Weight: {payload['weight_oz']:.2f} oz")
print(f"Estimated Volume: {payload['estimated_volume_in3']:.2f} in^3\n")

score = calculate_mission2_score(example_passengers, example_laps)
print("--- Score Calculation Example ---")
print(f"Net Income for {example_passengers} passengers and {example_laps} laps: {score:.2f}\n")

In [ ]:
def plot_score_contour(max_passengers=50, max_laps=12, battery_capacity_wh=100.0):
    """
    Generates and displays a 2D contour plot of Net Income vs. Laps and Passengers.
    """
    pax_range = np.arange(3, max_passengers + 1)
    laps_range = np.arange(1, max_laps + 1)

    X, Y = np.meshgrid(pax_range, laps_range)
    Z = np.zeros_like(X, dtype=float)

    for i in range(Y.shape[0]):  # Corresponds to laps
        for j in range(X.shape[1]):  # Corresponds to passengers
            pax = X[i, j]
            laps = Y[i, j]
            Z[i, j] = calculate_mission2_score(pax, laps, battery_capacity_wh)

    plt.figure(figsize=(12, 8))
    # Filled contour plot
    contour = plt.contourf(X, Y, Z, levels=20, cmap="viridis")

    # Add contour lines with labels
    line_contour = plt.contour(X, Y, Z, levels=contour.levels, colors="white", linewidths=0.5)
    plt.clabel(line_contour, inline=True, fontsize=8, fmt="%1.0f")

    plt.colorbar(contour, label="Net Income Score")
    plt.title(f"Mission 2 Net Income Contour Plot (Battery: {battery_capacity_wh}Wh)")
    plt.xlabel("Number of Passengers")
    plt.ylabel("Number of Laps")
    plt.grid(True, linestyle="--", alpha=0.6)
    plt.show()

In [ ]:
plot_score_contour(max_passengers=51, max_laps=12)

In [ ]:
def plot_payload_vs_pax(max_passengers=50):
    """
    Generates and displays a 2D plot of payload weight and volume vs. number of passengers.
    """
    pax_range = np.arange(3, max_passengers + 1)
    weights_oz = []
    volumes = []
    for pax in pax_range:
        props = calculate_payload_properties(pax)
        weights_oz.append(props["weight_oz"])
        volumes.append(props["estimated_volume_in3"])

    # Convert weights from ounces to pounds
    weights_lb = np.array(weights_oz) / 16.0

    fig, ax1 = plt.subplots(figsize=(12, 7))

    # Plot weight on the primary y-axis
    color = "tab:red"
    ax1.set_xlabel("Number of Passengers")
    ax1.set_ylabel("Total Payload Weight (lb)", color=color)
    ax1.plot(pax_range, weights_lb, color=color, marker="o", linestyle="-", label="Weight (lb)")
    ax1.tick_params(axis="y", labelcolor=color)
    ax1.grid(True, which="both", linestyle="--", linewidth=0.5)

    # Create a second y-axis for volume that shares the same x-axis
    ax2 = ax1.twinx()
    color = "tab:blue"
    ax2.set_ylabel("Estimated Payload Volume (in³)", color=color)
    ax2.plot(pax_range, volumes, color=color, marker="x", linestyle="--", label="Volume (in³)")
    ax2.tick_params(axis="y", labelcolor=color)

    plt.title("Payload Weight and Volume vs. Number of Passengers")
    fig.tight_layout()  # Adjust layout to make room for y-labels
    plt.show()

In [ ]:
plot_payload_vs_pax(max_passengers=51)

# Turning

## Bank angle and structural loading (g-loading $n$)
in a coordinated turn

$$R = \frac{v^2}{g \dot \tan \theta} = \frac{v^2}{g \sqrt{n^2 - 1}}$$
$$t = \frac{2 \pi v}{g \sqrt{n^2 - 1}}$$

In [ ]:
inch_to_m = 0.0254  # [=] m/inch
feet_to_m = 0.3048  # [=] m/ft
lb_to_kg = 0.453592  # [=] kg/lb
g = 9.81  # [=] m/s^2

bank_angles = np.linspace(1, 80, 80)
g_loadings = 1 / np.cos(np.deg2rad(bank_angles))

v = 100 * feet_to_m
Rs = v**2 / g / np.tan(np.deg2rad(bank_angles))
turning_time = 2 * np.pi * Rs / v

# Make two y axes
fig, g_load_axis = plt.subplots()
g_load_axis.plot(bank_angles, g_loadings, label="G loading", color="tab:orange")
g_load_axis.set_xlabel("Bank angle (°)")
g_load_axis.set_ylabel("G Loading")

turn_radius_axis = g_load_axis.twinx()
turn_radius_axis.plot(bank_angles, Rs, label="Turn Radius")
turn_radius_axis.set_ylabel("Turn radius")
turn_radius_axis.set_ylim([0, 500])

# plt.plot(bank_angles, g_loadings, label= "G Loading")
# plt.plot(bank_angles, Rs)
# plt.plot(bank_angles, Rs)

# plt.xlabel('Bank angle [deg]')
# plt.ylabel('g loading')
fig.legend()
plt.show()

In [ ]:
g_loading_max = 4
speeds = np.linspace(10, 500, 100) * feet_to_m
turning_times = 2 * np.pi * speeds / g / np.sqrt(g_loading_max**2 - 1)
lap_times = 1000 * 2 * feet_to_m / speeds + turning_times

# make two y axis
fig, ax1 = plt.subplots()
ax1.plot(speeds / feet_to_m, lap_times)
ax1.set_xlabel("Speed [ft/s]")
ax1.set_ylabel("Lap time [s]")


ax2 = ax1.twinx()
ax2.plot(speeds / feet_to_m, 300 / lap_times, color="tab:orange")
ax2.set_ylabel("Laps in 5 minutes")
plt.show()
